# 07 - Gradient Boost
---

### **Introduction**
For random forests, we trained many decision trees to predict the target and then aggregated their predictions. Gradient boosting is a similar idea in that it works by training many decision trees. However it differs to random forests by what each tree predicts and how they are combined. In particular, during gradient boosting, each decision tree does not aim to predict the target but rather it aims to predict the errors made by the previous iteration of the model. Gradient boosting is an example of **additive modelling** whereby models are trained sequentially. Each model's prediction is added on to the previous model's prediction, usually after being scaled first. 

To explain gradient boosting we will start with a concrete example of how we would use it to predict a continuous target using the RSS as the loss function. Note that this is called residual boosting. We will then take a more abstract view and explain the concept of gradient boosting more generally before demonstrating how residual boosting is a specific implementation of the wider gradient boosting technique. 

### **Residual Boosting - Concrete Example**
Suppose we want to predict a continuous target. The steps to residual boosting are as follows: 

1. Fit a decision tree to the training data; recall we use the RSS to decide which variables and thresholds/values to use at each node.
2. Use the model to make a prediction $\hat{y}_i$ about each observation in the training data
3. Compute the residual for each prediction: $r_i = y_i - \hat{y}_i$
4. Fit another decision tree to predict the resduals from the model $\hat r_i$
5. Combine the prediction from the model with the prediction from the new decision tree via a learning rate ($\lambda$): $\hat{y}_i + \lambda \hat r_i$
5. Repeat steps 2 - 5, each time using the combined model to make a prediction about the target, computing the residuals and then fitting another tree to predict the residuals before adding this prediction on, scaled by the learning rate. Stop after a certain number of trees have been trained or the step size (learning rate * prediction) of the final tree is below a  set threhsold. 

This method allows us to iteratively correct the mistakes made by the model. 

We can write the prediction $F_M(x)$ from the combined boosted model after $M$ iterations as follows:

$F_M(x) = \sum_{k=1}^{M}\gamma_k T_k(x)$

where 
$\gamma_k$: Weight (often absorbed into learning rate)
$T_k(x)$: Tree at iteration k

The loss function $\mathcal{L}$ (i.e. RSS) can then be written as $\mathcal{L} = \sum_{i=1}^n (y_i - F(x_i))^2$ where $F$ is the final model after all iterations. 

### **Gradient Boosting - Abstract Discussion**
Consider a loss-function $\mathcal{L}$ which is a function of a model $F(x_i)$ and define the **pseudo-residuals** as $\hat r_i = -\frac{\partial \mathcal{L}}{\partial F(x_i)}$
In standard gradient descent, parameters are updated via $\theta \leftarrow \theta - \eta \nabla_\theta \mathcal{L}$
In boosting, we update the model $F(x_i)$ function iteratively so that after $m$ iterations the model is defined $F_m(x_i) = F_{m-1}(x_i) -  \eta \frac{\partial \mathcal{L}}{\partial F(x_i)}$
We can write this in terms of the pseudo-residuals as $F_m(x_i) = F_{m-1}(x_i) - \eta \hat r_i$
So gradient boosting is a kind of functional version of gradient descent where we minimise the loss function by changing the model.

### **Linking the Concrete + Abstract**
For the residual boosting example, the loss function (RSS) $\mathcal{L}$ can be written as $\mathcal{L} = \sum_{i=1}^n (y_i - F(x_i))^2$ where $F$ is the final model after all iterations. 

If we compute the pseudo-residuals we get:
$\hat r_i = \frac{\partial \mathcal{L}}{\partial F(x_i)} = \frac{\partial}{\partial F(x_i)}(y_i^2 -2y_iF(x_i) + F(x_i)^2) = -2y_i + 2F(x_i) = -2 \left(y_i - F(x_i)\right)$

Notice how, up to the constant $-2$, the pseudo-residuals are exactly the residuals $r_i = y_i - F(x_i)$

In the example we were creating a new iteraiton of the model by predicting the residuals, multiplying it by a learning rate and then adding it to the predictions from the previous iteration of the model. More concretely:
$F_m(x_i) = F_{m-1}(x_i) + \lambda \hat \varepsilon_i$ where $\varepsilon_i$ are the predicted residuals from the previous model

Since, up to the constant, the predicted residual are exactly pseudo-residuals, what we are actually doing in residual boosting is a special case of gradient boosting. In particular, where we are predicting a continuous target and fitting the decision trees using RSS.  

### **Extending this Idea**
The gradient boosting idea can be extended to other forms of loss function beyond just the one we derived for RSS. The only requirement is that the loss function is differentiable with respect to the model predictions. For instance, suppose we are attempting to predict a binary target. In this case we might define the logistic loss:

$\mathcal{L} = \sum_{i=1}^n \log\left(1 + e^{-y_i F(x_i)}\right)$

We can compute the pseudo-residuals as: $\hat r_i = - \frac{\partial \mathcal{L}}{\partial F(x_i)}$

These pseudo-residuals no longer correspond to simple residuals $y_i - F(x_i)$. Instead, they represent the direction in which the model predictions must change in order to reduce the loss most rapidly. We then fit a regression tree to these pseudo-residuals and update the model:

$F_m(x_i) = F_{m-1}(x_i) + \eta \hat r_i$ 

where $\eta$ is the learning rate.

Thus, gradient boosting provides a general framework for minimising arbitrary differentiable loss functions by iteratively fitting weak learners to the negative gradient of the loss function. Residual boosting is simply the special case obtained when the loss function is the residual sum of squares.